In [2]:
#!pip install prophet

import pandas as pd
from prophet import Prophet

df = pd.read_csv('final_data.csv', delimiter=",", encoding='ISO-8859-1')

# ensuring Purchase_date is of the right type
df['purchase_date'] = pd.to_datetime(df['purchase_date'])

# note: we chose prophet model among others because not only is it very interpretable and user-friendly,
#       but it can handle data which displays trends and seasonality, which is commonly observed in sales data.

# function to apply Prophet model for each group
def forecast_sales(group):
    # Prepare the data for Prophet
    sales_data = group.groupby('purchase_date')['quantity_purchased'].sum().reset_index()
    sales_data.columns = ['ds', 'y']  # must be 'ds' for date and 'y' for target to meet prophet requirements

    # Initialize and fit the Prophet model
    model = Prophet()
    model.fit(sales_data)

    # predict next 30 days/1 month
    forecast = model.predict(model.make_future_dataframe(periods=30))

    # Return the forecast and the sum of future sales (demand forecast for EOQ)
    forecasted_demand = forecast['yhat'].sum()  # Total forecasted demand
    return forecasted_demand

# Group by 'item_name', 'store_region', 'supplier' combi, then apply the forecast_sales function
grouped_forecasts = df.groupby(['item_name', 'store_region', 'supplier']).apply(forecast_sales).reset_index()
grouped_forecasts.columns = ['item_name', 'store_region', 'supplier', 'forecasted_demand']

# preview
print(grouped_forecasts.head())


C:\Users\covan\AppData\Local\Temp\ipykernel_34744\3541349224.py:6: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_data.csv', delimiter=",", encoding='ISO-8859-1')
15:58:07 - cmdstanpy - INFO - Chain [1] start processing
15:58:07 - cmdstanpy - INFO - Chain [1] done processing
15:58:07 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
15:58:07 - cmdstanpy - INFO - Chain [1] start processing
15:58:07 - cmdstanpy - INFO - Chain [1] done processing
15:58:07 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


RuntimeError: Error during optimization! Command 'C:\Users\covan\Downloads\dsa3101\venv\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=48106 data file=C:\Users\covan\AppData\Local\Temp\tmpiog18bus\82i9a704.json init=C:\Users\covan\AppData\Local\Temp\tmpiog18bus\t60_9rhn.json output file=C:\Users\covan\AppData\Local\Temp\tmpiog18bus\prophet_model0p1domiv\prophet_model-20241115155807.csv method=optimize algorithm=newton iter=10000' failed: 

In [ ]:
import numpy as np
# Economic order quantity (EOQ) is a formula that helps businesses determine the ideal order size for inventory to minimize costs and meet demand
# Define a function to calculate EOQ based on forecasted demand
def calculate_eoq(row):
    # extract the forecasted demand
    D = row['forecasted_demand']

    # corresponding unit price and inventory cost for the item_name, store_region and supplier
    S = df.loc[(df['item_name'] == row['item_name']) & (df['supplier'] == row['supplier']) & (df['store_region'] == row['store_region']), 'unit_price'].values[0]
    H = df.loc[(df['item_name'] == row['item_name']) & (df['supplier'] == row['supplier']) & (df['store_region'] == row['store_region']), 'inventory_cost'].values[0]

    # calculate EOQ using the standard EOQ formula
    eoq = np.sqrt((2*S*D)/H)
    return eoq

# Apply the EOQ calculation to each product-store_region-supplier group
grouped_forecasts['EOQ'] = grouped_forecasts.apply(calculate_eoq, axis=1)

# forecasted demand and EOQ preview
print(grouped_forecasts.head())


In [ ]:
# Join the current inventory level with the EOQ data
grouped_data = pd.merge(grouped_forecasts, df[['item_name', 'store_region', 'supplier', 'inventory_level']].drop_duplicates(),
                        on=['item_name', 'store_region', 'supplier'], how='left')

# Calculate reorder quantity if forecasted demand exceeds current inventory
grouped_data['reorder_quantity'] = grouped_data.apply(
    lambda row: row['EOQ'] if row['forecasted_demand'] > row['inventory_level'] else 0, axis=1)

grouped_data = grouped_data.drop(columns=['inventory_level'])

results = grouped_data.groupby(['item_name', 'store_region', 'supplier', 'forecasted_demand', 'EOQ', 'reorder_quantity'], as_index=False).first()

# Preview
print(results.head())
